# Imports

In [1]:
import pandas as pd
import numpy as np

import os

from glob import glob
from tqdm import tqdm


In [2]:
from data_transformation import CleanData
from features_library import (daily_returns, daily_volatility,
                              daily_log_returns, daily_squared_returns, 
                              monthly_realized_volatility, weekly_realized_volatility, 
                              relative_price_range, money_flow_index, 
                              avg_directional_movement_index, williams_accumulation_distribution)

# Load Data

In [3]:
DIR_CRYPTO_DATA = "./data/1_Crypto_data/"
DIR_FINANCIAL_DATA = "./data/1_Financial_component_data/"


In [4]:
dict_assets = {}
for file_crypto in glob(f"{DIR_CRYPTO_DATA}*.csv"):

    # Read CSV
    asset_name = os.path.basename(file_crypto).split(".")[0]
    df_asset = pd.read_csv(file_crypto)

    # Pre-Process Data
    preproc = CleanData(df_asset)
    preproc.make_datetime_index(date_col='Date')
    preproc.resample(freq='1D')
    preproc.strip_column_name()

    # assign to dict
    dict_assets[asset_name] = preproc.df


In [5]:
dict_fd = {}
for file_fd in glob(f"{DIR_FINANCIAL_DATA}*.csv"):

    # Read CSV
    fd_name = os.path.basename(file_fd).split(".")[0]
    df_fd = pd.read_csv(file_fd)

    # Pre-Process Data
    preproc = CleanData(df_fd)
    preproc.make_datetime_index(date_col='Date')
    preproc.resample(freq='1D')
    preproc.strip_column_name()

    # Keep??
    # if ('Adj Close' not in df_fd.columns) and ('Close' in df_fd.columns):
    #     df_fd['Adj Close'] = df_fd['Close'].copy()

    # assign to dict
    dict_fd[fd_name] = preproc.df


# Feature Engineering

In [ ]:
dict_features =


In [7]:
preproc.df.columns


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [8]:
daily_returns(dict_assets['AVAX-USD']['Open'])


2020-09-22         NaN
2020-09-23    0.067158
2020-09-24   -0.282568
2020-09-25    0.208517
2020-09-26    0.021610
                ...   
2023-02-24   -0.030262
2023-02-25   -0.067448
2023-02-26   -0.022647
2023-02-27    0.027038
2023-02-28   -0.027326
Freq: D, Name: Open, Length: 890, dtype: float64